<a href="https://colab.research.google.com/github/YokoyamaLab/PythonBasics/blob/2024/day04_q2FaceComponents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day04 課題Q2 アバターに使う顔のパーツを描こう

In [ ]:
#@title 学籍番号、名前
id = "" #@param {type:"string"}
name = "" #@param {type:"string"}

### 課題説明

この課題ではこれまでに作った顔のパーツを描くクラスの一部を拡張(継承)して、皆さんの自画像を描いてもらいます。

解説は長くなるので講義資料と共に別ファイルで配布しています。まずはそちらを見てください。

### 顔のパーツの実装

まずComponentクラスや各パーツの基本となるクラスをGoogle Driveからimportします。

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

# Gdrive上からGoogle Colabの環境にダウンロードする
!cp "/content/gdrive/Shareddrives/2024-35A01／情報科学 e／横山 昌平/lib/face_component.py" "/content"

# ダウンロードしたものを呼び出す
from face_component import Component,Face,Eyes,Ears,Mouth,Nose,Hair,SmoothHair

# 必要ライブラリのインポート
from PIL import Image, ImageDraw, ImageFont
import math

では、講義で皆さんに割り当てた顔のパーツを3つ実装してください。

### 顔のパーツ（輪郭）

輪郭を表すFaceクラスを継承して独自の輪郭を描くクラスを作成してください。インスタンス変数のfeatureには、そのパーツを端的に表すタイトルを付けてください。

In [ ]:
# コードはここへ

### 顔のパーツ（髪型）

髪の毛を表すHairクラスを継承して独自の輪郭を描くクラスを作成してください。インスタンス変数のfeatureには、そのパーツを端的に表すタイトルを付けてください。

In [ ]:
# コードはここへ

### 顔のパーツ（指定パーツ）

指定された顔のパーツを指定されたクラス名で定義してください。インスタンス変数のfeatureには、そのパーツを端的に表すタイトルを付けてください。

In [ ]:
# コードはここへ

## 顔のパーツ（任意パーツ）

上記で描いていないパーツを一つ選びそれを描くクラスを指定された番号を末尾につけたクラス名で(指定番号が12でEyesクラスを作るならEyes12)作成してください。

In [ ]:
# コードはここへ

### 検証用コード

ここで実装したパーツ以外は、講義ノートで作成したクラスを使って構いません。クラス定義はすでにインポートされていますので、クラス名を指定すれば使えます。

In [ ]:
# 検証用の描画コードはここへ

必ず検証用コードで自分の実装したコードを利用した顔が、想定通り描けている事を確認してから、提出してください。どんなものを実装したか確認しやすいので、顔を描画した状態で提出してください。